## Modèle avancé

In [1]:
from helpers import *

In [2]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [3]:
cols = ['sentiment', 'timestamp', 'date', "query", "username", "comment"]  
df = pd.read_csv("./../input/training.1600000.processed.noemoticon.csv", header=None, names=cols)

# df = df.sample(n=100000, random_state=42)
len_df = len(df)
print(df.shape)
df.head()

(1600000, 6)


,sentiment,timestamp,date,query,username,comment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df = clean_df(df)
print(df.shape)
df.head()

Processing: 100%|██████████| 1600000/1600000 [00:02<00:00, 573022.59it/s]


(1469990, 5)


,comment,sentiment,comment_clean,words_nb,words_nb_clean
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,awww thats a bummer you shoulda got david carr...,19,16
1,is upset that he can't update his Facebook by ...,0,is upset that he cant update his facebook by t...,21,21
2,@Kenichan I dived many times for the ball. Man...,0,i dived many times for the ball managed to sav...,18,16
3,my whole body feels itchy and like its on fire,0,my whole body feels itchy and like its on fire,10,10
4,"@nationwideclass no, it's not behaving at all....",0,no its not behaving at all im mad why am i her...,21,20


### Word2Vec

In [5]:
df_w2vec = df.copy()

config = {
    "vector_size": 100,
    # Set input length to be the max from the number of words
    "input_length": df_w2vec["words_nb_clean"].max(),
    # Word before & after for word2vec
    "window": 5,
    # Max number of workers
    "workers": 8,
    # Vocal length
    "vocab_length": 70000
}

(model_w2vec, tokenizer_w2vec, callbacks_w2vec, X_w2vec, y_w2vec) = get_w2vec_data(config, df_w2vec)


W2vec Vocabulary Length (you can adjust the vocal length): 44901
Embedding Matrix Shape: (70000, 100)


2024-02-07 12:16:37.862436: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-07 12:16:37.862462: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2024-02-07 12:16:37.862474: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2024-02-07 12:16:37.862519: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-07 12:16:37.862537: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model_w2vec.summary()

Model: "Sentiment_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 100)           7000000   
                                                                 
 bidirectional (Bidirection  (None, 40, 200)           160800    
 al)                                                             
                                                                 
 conv1d (Conv1D)             (None, 36, 100)           100100    
                                                                 
 global_max_pooling1d (Glob  (None, 100)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 16)                1616      
                                                                 
 dense_1 (Dense)             (None, 1)             

In [7]:
space = {
    "input_length": config["input_length"],
    'experiment_name': f"Tweet Sentiment - {len_df} - w2vec",
    'batch_size': hp.choice('batch_size', [128, 256]),  
    'epochs': hp.choice('epochs', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),  
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.001),  
}

In [8]:
trials = Trials()
fmin(  
    fn=lambda params: objective(params, X_w2vec, y_w2vec, model_w2vec, callbacks_w2vec, tokenizer_w2vec),  
    space=space,  
    algo=tpe.suggest,  
    max_evals=10,
    trials=trials  
)

Running one fit with the params:                      
{'batch_size': 128, 'epochs': 11, 'experiment_name': 'Tweet Sentiment - 1600000 - w2vec', 'input_length': 40, 'learning_rate': 0.00047031010256646654}
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024/02/07 12:16:38 INFO mlflow.tracking.fluent: Experiment with name 'Tweet Sentiment - 1600000 - w2vec' does not exist. Creating a new experiment.



Epoch 1/11                                            

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024-02-07 12:16:39.394251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


7236/7236 [==============================] - 512s 71ms/step - loss: 0.4464 - accuracy: 0.7897 - val_loss: 0.4152 - val_accuracy: 0.8098 - lr: 4.7031e-04

Epoch 2/11                                            

7236/7236 [==============================] - 595s 82ms/step - loss: 0.4133 - accuracy: 0.8093 - val_loss: 0.4003 - val_accuracy: 0.8165 - lr: 4.7031e-04

Epoch 3/11                                            

  0%|          | 0/10 [30:51<?, ?trial/s, best loss=?]

In [ ]:

with open("./model.pkl", "rb") as file:  
    model = pickle.load(file)
    
with open("./tokenizer.pkl", "rb") as file:  
    vector_model = pickle.load(file)

params = json.load(open("./params.json", 'r', encoding='utf-8'))
def pad_sequences(sequence, maxlen, value=0):  
    return np.array([np.pad(s[:maxlen], (max(0, maxlen-len(s)), 0), 'constant', constant_values=value) if len(s) < maxlen else s[:maxlen] for s in sequence])  
  
def predict(comment, model, vector_model):
    # We need to pad sequences to ensure uniform input size  
    sequence = tokenizer_w2vec.texts_to_sequences(comment)
    padded_sequence = pad_sequences(sequence, maxlen=int(params["input_length"]))

    prediction = model.predict(padded_sequence)
    # print(prediction)
    sentiment = prediction.astype(float)[0][0]
    return sentiment.item()
    
def comment_to_vec(comment, model):
    vec = np.zeros(100)
    num_words = 0
    for word in comment:
        if word in model.wv:
            vec += model.wv[word]
            num_words += 1
    if num_words > 0:
        vec /= num_words
    return vec

with tf.device('/CPU:0'):
    print(predict(["I am so sad, this is very bad news, terrible!"], model, tokenizer_w2vec))
    print(predict(["I am so happy, this is very good news, congrats!"], model, tokenizer_w2vec))


In [ ]:
# We need to pad sequences to ensure uniform input size  
sequence = tokenizer_w2vec.texts_to_sequences(df["comment_clean"])
padded_sequence = pad_sequences(sequence, maxlen=int(params["input_length"]))

prediction = model.predict(padded_sequence)

In [ ]:
y_pred_rounded = (prediction > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)
accuracy = accuracy_score(df["sentiment"], y_pred_rounded)

### Glove

In [ ]:
df_glove = df.copy()

config = {
    # Best to use 100 since I'm using glove 100 of pretrained data
    "vector_size": 100,
    # Set input length to be the max from the number of words
    "input_length": df_glove["words_nb_clean"].max(),
    # Word before & after for word2vec
    "window": 5,
    # Max number of workers
    "workers": 8,
    # Vocal length
    "vocab_length": 70000
}

(model_glove, tokenizer_glove, callbacks_glove, X_glove, y_glove) = get_glove_data(config, df_glove)


In [ ]:
model_glove.summary()

In [ ]:
space = {
    "input_length": config["input_length"],
    'experiment_name': f"Tweet Sentiment - {len_df} - glove",
    'batch_size': hp.choice('batch_size', [128, 256]),  
    'epochs': hp.choice('epochs', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),  
    'learning_rate': hp.uniform('learning_rate', 0.0001, 0.001),  
}

In [ ]:
trials = Trials()
fmin(  
    fn=lambda params: objective(params, X_glove, y_glove, model_glove, callbacks_glove, tokenizer_glove),  
    space=space,  
    algo=tpe.suggest,
    max_evals=10,
    trials=trials  
)